# Using Agents for Amazon Bedrock

In this notebook we will create an Agent for Amazon Bedrock that interacts with some synthetic data held in an SQLite database.

With this agent, tennis club members can check availabilty and book the tennis courts at the club.

A Lambda function is used to check court availabilty and create bookings.

## Prerequisites
Update the botocore and boto3 packages 

In [ ]:
%pip install --upgrade -q botocore
%pip install --upgrade -q boto3
%pip install --upgrade -q awscli

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging

In [ ]:
# Logging facility for Python is used to help confirm things are working as expected!
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Create the boto3 clients needed for a number of AWS services

In [ ]:
# creating boto3 clients for the services we'll use
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Setting configuration variables for the session, as well as those needed for the Bedrock agent and a Lambda function that will be created later

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

In [ ]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "booking-asst-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for booking tennis courts"
agent_instruction = "You are a polite booking system agent, helping helping members of a tennis club to book tennis courts"
agent_action_group_name = "BookingsActionGroup"
agent_action_group_description = "Actions for helping members of tennis club to book tennis courts"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Creating a Lambda Function

A Lambda function is going to interact with our data in SQLite. This function will later be triggered by the agent.
1. `lambda_function.py` file contains the function logic
2. An IAM role is needed to give our Lambda function the required permissions 

## Creating a Dataset

First we create the `tennis_booking_system.db` file that contains a database populated with some generated data for our application. 
SQLite is a Python library that provides a lightweight SQL compatible database, we can use it to prototype our application. 

In [ ]:
# Tennis court booking database for use by a Lambda function
import sqlite3
import random
from datetime import date, timedelta, datetime

# Connect to the SQLite database and create a new one if it doesn't already exist
conn = sqlite3.connect('tennis_booking_system.db')
c = conn.cursor()

# Create courts table
c.execute('''
    CREATE TABLE IF NOT EXISTS courts (
        court_id INTEGER PRIMARY KEY AUTOINCREMENT,
        court_name TEXT UNIQUE
    )
''')

# Insert the 4 tennis courts (Court A, Court B, Court C, and Court D)
courts = [("Court A",), ("Court B",), ("Court C",), ("Court D",)]
c.executemany("INSERT OR IGNORE INTO courts (court_name) VALUES (?)", courts)

# Create bookings table
c.execute('''
    CREATE TABLE IF NOT EXISTS bookings (
        booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
        court_id INTEGER,
        booking_date TEXT,  -- e.g., '2025-02-15'
        start_time TEXT,    -- e.g., '09:00'
        end_time TEXT,      -- e.g., '10:00'
        booked_by TEXT,     -- Name of the person booking the court
        notes TEXT,
        FOREIGN KEY(court_id) REFERENCES courts(court_id)
    )
''')

# Generate random booking data for 10 bookings
customer_names = [
    'Frank Watson', 'Anna Miller', 'Anand Shah', 'Denise Washington',
    'Tina Adams', 'Nathan Sharma', 'Sam Green', 'Maria Hernandez',
    'James Smith', 'Carlos da Silva'
]

for i in range(10):
    # Choose a random court from the four available
    chosen_court = random.choice(["Court A", "Court B", "Court C", "Court D"])
    c.execute("SELECT court_id FROM courts WHERE court_name = ?", (chosen_court,))
    court_id = c.fetchone()[0]
    
    # Generate a random booking date within the next 30 days
    random_days = random.randint(0, 30)
    booking_date = (date.today() + timedelta(days=random_days)).strftime('%Y-%m-%d')
    
    # Generate a random start time between 8:00 and 19:00 (to allow a 1- or 2-hour slot)
    start_hour = random.randint(8, 19)
    start_minute = random.choice([0, 15, 30, 45])
    # Use a dummy date to format the time
    start_time_dt = datetime(2000, 1, 1, start_hour, start_minute)
    # Randomly choose a booking duration of 1 or 2 hours
    duration = random.choice([1, 2])
    end_time_dt = start_time_dt + timedelta(hours=duration)
    
    start_time_str = start_time_dt.strftime('%H:%M')
    end_time_str = end_time_dt.strftime('%H:%M')
    
    booked_by = random.choice(customer_names)
    notes = "Random booking"  # Optionally, add more details
    
    c.execute('''
        INSERT INTO bookings (court_id, booking_date, start_time, end_time, booked_by, notes)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (court_id, booking_date, start_time_str, end_time_str, booked_by, notes))

# Commit the changes and close the connection
conn.commit()
conn.close()


Create our lambda function. 
It has the ability to check which courts are available at a given time and date, and book courts if no conflicts exist.

In [ ]:
%%writefile lambda_function.py
import os
import shutil
import sqlite3
from datetime import datetime

def check_court_availability(court_name, booking_date, start_time, end_time):
    """
    Check if the specified court is available on a given date and time slot.
    Returns "Available" if free, otherwise returns a message with conflict details.
    """
    # Connect to the tennis booking database
    conn = sqlite3.connect('/tmp/tennis_booking_system.db')
    c = conn.cursor()
    
    # Get the court ID from the courts table
    c.execute("SELECT court_id FROM courts WHERE court_name = ?", (court_name,))
    court = c.fetchone()
    if not court:
        conn.close()
        return f"Court {court_name} not found."
    court_id = court[0]
    
    # Retrieve any existing bookings for the court on the given date
    c.execute("SELECT start_time, end_time FROM bookings WHERE court_id = ? AND booking_date = ?", (court_id, booking_date))
    existing_bookings = c.fetchall()
    conn.close()
    
    # Convert the times to datetime objects for easy comparison
    new_start = datetime.strptime(start_time, '%H:%M')
    new_end = datetime.strptime(end_time, '%H:%M')
    
    conflicts = []
    for booking in existing_bookings:
        booked_start = datetime.strptime(booking[0], '%H:%M')
        booked_end = datetime.strptime(booking[1], '%H:%M')
        # Check if the new booking overlaps with any existing booking
        if new_start < booked_end and new_end > booked_start:
            conflicts.append(f"{booking[0]}-{booking[1]}")
    
    if conflicts:
        return f"Not available. Conflicts with booking(s) at: {', '.join(conflicts)}"
    else:
        return "Available"

def reserve_court(court_name, booking_date, start_time, end_time, booked_by, notes=""):
    """
    Reserve the specified court on a given date and time slot for a person.
    First checks for availability and then inserts the booking if possible.
    """
    # Connect to the tennis booking database
    conn = sqlite3.connect('/tmp/tennis_booking_system.db')
    c = conn.cursor()
    
    # Get the court ID from the courts table
    c.execute("SELECT court_id FROM courts WHERE court_name = ?", (court_name,))
    court = c.fetchone()
    if not court:
        conn.close()
        return f"Court {court_name} not found."
    court_id = court[0]
    
    # Check availability for the requested slot
    c.execute("SELECT start_time, end_time FROM bookings WHERE court_id = ? AND booking_date = ?", (court_id, booking_date))
    existing_bookings = c.fetchall()
    
    new_start = datetime.strptime(start_time, '%H:%M')
    new_end = datetime.strptime(end_time, '%H:%M')
    
    for booking in existing_bookings:
        booked_start = datetime.strptime(booking[0], '%H:%M')
        booked_end = datetime.strptime(booking[1], '%H:%M')
        if new_start < booked_end and new_end > booked_start:
            conn.close()
            return f"Cannot reserve {court_name} on {booking_date} from {start_time} to {end_time}. Conflicts with booking {booking[0]}-{booking[1]}."
    
    # Insert the new booking since there is no conflict
    c.execute("""
        INSERT INTO bookings (court_id, booking_date, start_time, end_time, booked_by, notes)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (court_id, booking_date, start_time, end_time, booked_by, notes))
    
    conn.commit()
    conn.close()
    return f"Booking confirmed for {court_name} on {booking_date} from {start_time} to {end_time} for {booked_by}."

def lambda_handler(event, context):
    """
    Lambda handler that supports two functions:
      - 'check_court_availability': Checks if a specified court is available.
      - 'reserve_court': Attempts to reserve a specified court.
    The event payload should include a 'function' field and parameters.
    """
    original_db_file = 'tennis_booking_system.db'
    target_db_file = '/tmp/tennis_booking_system.db'
    if not os.path.exists(target_db_file):
        shutil.copy2(original_db_file, target_db_file)
    
    actionGroup = event.get('actionGroup')
    function = event.get('function')
    parameters = event.get('parameters', [])
    responseBody = {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }
    
    if function == 'check_court_availability':
        court_name = None
        booking_date = None
        start_time = None
        end_time = None
        
        for param in parameters:
            if param["name"] == "court_name":
                court_name = param["value"]
            elif param["name"] == "booking_date":
                booking_date = param["value"]
            elif param["name"] == "start_time":
                start_time = param["value"]
            elif param["name"] == "end_time":
                end_time = param["value"]
        
        # Validate parameters
        if not court_name:
            raise Exception("Missing mandatory parameter: court_name")
        if not booking_date:
            raise Exception("Missing mandatory parameter: booking_date")
        if not start_time:
            raise Exception("Missing mandatory parameter: start_time")
        if not end_time:
            raise Exception("Missing mandatory parameter: end_time")
        
        availability = check_court_availability(court_name, booking_date, start_time, end_time)
        responseBody = {
            'TEXT': {
                "body": f"Availability for {court_name} on {booking_date} from {start_time} to {end_time}: {availability}"
            }
        }
    
    elif function == 'reserve_court':
        court_name = None
        booking_date = None
        start_time = None
        end_time = None
        booked_by = None
        notes = ""
        
        for param in parameters:
            if param["name"] == "court_name":
                court_name = param["value"]
            elif param["name"] == "booking_date":
                booking_date = param["value"]
            elif param["name"] == "start_time":
                start_time = param["value"]
            elif param["name"] == "end_time":
                end_time = param["value"]
            elif param["name"] == "booked_by":
                booked_by = param["value"]
            elif param["name"] == "notes":
                notes = param["value"]
        
        # Validate parameters
        if not court_name:
            raise Exception("Missing mandatory parameter: court_name")
        if not booking_date:
            raise Exception("Missing mandatory parameter: booking_date")
        if not start_time:
            raise Exception("Missing mandatory parameter: start_time")
        if not end_time:
            raise Exception("Missing mandatory parameter: end_time")
        if not booked_by:
            raise Exception("Missing mandatory parameter: booked_by")
        
        completion_message = reserve_court(court_name, booking_date, start_time, end_time, booked_by, notes)
        responseBody = {
            'TEXT': {
                "body": completion_message
            }
        }
    
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }
    
    function_response = {
        'response': action_response,
        'messageVersion': event.get('messageVersion')
    }
    
    print("Response:", function_response)
    return function_response

Next let's create the Lambda IAM role and policy to invoke a Bedrock model

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

We can now package the Lambda function to a Zip file and create the Lambda function using boto3

In [ ]:
# Package up the Lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("tennis_booking_system.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [ ]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

### Creating the agent
Once the needed IAM role is created, we use the Bedrock Agent client `create_agent` function to create the agent. 
It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. 

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

Store the agent id in a local variable to use later

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## Create Agent Action Group
We will now create an agent action group that uses the Lambda function. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

The Action Group functionality is defined using a `functionSchema` and we need to provide the `name`, `description` and `parameters` for each agent function.

In [ ]:
agent_functions = [
    {
        'name': 'check_court_availability',
        'description': 'Check if a tennis court is available on a specific date and time slot',
        'parameters': {
            "court_name": {
                "description": "The name of the tennis court (e.g., 'Court A')",
                "required": True,
                "type": "string"
            },
            "booking_date": {
                "description": "The date for the booking in YYYY-MM-DD format",
                "required": True,
                "type": "string"
            },
            "start_time": {
                "description": "The start time of the booking in HH:MM format",
                "required": True,
                "type": "string"
            },
            "end_time": {
                "description": "The end time of the booking in HH:MM format",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'reserve_court',
        'description': 'Reserve a tennis court for a specific date and time slot',
        'parameters': {
            "court_name": {
                "description": "The name of the tennis court to reserve (e.g., 'Court A')",
                "required": True,
                "type": "string"
            },
            "booking_date": {
                "description": "The date for the booking in YYYY-MM-DD format",
                "required": True,
                "type": "string"
            },
            "start_time": {
                "description": "The start time of the booking in HH:MM format",
                "required": True,
                "type": "string"
            },
            "end_time": {
                "description": "The end time of the booking in HH:MM format",
                "required": True,
                "type": "string"
            },
            "booked_by": {
                "description": "The name of the person reserving the court",
                "required": True,
                "type": "string"
            }
        }
    }
]


In [ ]:
# Wait for the agent to finish being created
time.sleep(30)
# Configure and create the action group
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response

## Enable Agent to invoke the Action Group Lambda function
Enable the agent to invoke the Lambda function by adding the required permission to a resource-based policy. 

In [ ]:
# Allow invoke permission on Lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [ ]:
response

## Preparing the Agent

Creating a DRAFT version of the agent that can be used for internal testing.


In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## Invoke Agent

Invoking the agent using the `bedrock-agent-runtime` client

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Which courts are available on Thursday 2025-02-13 from 14:00 to 15:00?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Please book Court A on Thursday 2025-02-13 from 14:00 to 15:00 for Faye Ellis",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Please book Court A on Thursday from 14:00 to 15:00 for Faye Ellis",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

In [ ]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

## Cleaning Up

Run the following cells to delete everything we created, including:

1. Action group
2. Agent 
3. Lambda function
4. IAM roles and policies


In [ ]:
# Deleting the Action Group
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
# Deleting the Agent
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Deleting the Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

In [ ]:
# Deleting the IAM Roles
for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


## Remember to also delete the Jupyter Notebook if you no longer need it!